In [1]:
!pip install cirq

In [2]:
import cirq
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from scipy.linalg import eigh

In [3]:
def load_and_scale_iris():
    """
    アイリスデータセットを読み込み、標準化して返します。
    """
    data = load_iris()
    iris_df = pd.DataFrame(data.data, columns=data.feature_names)
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(iris_df)
    return scaled_features  # 全てのサンプルを使用

In [4]:
def create_quantum_encoding_circuit(qubits, data):
    """
    データを量子状態にエンコードするための回路を作成します。
    """
    circuit = cirq.Circuit()
    for i, value in enumerate(data):
        # 振幅エンコーディングとしてRyゲートを使用
        # 値の範囲を -1 から 1 の間にクリップして arccos に適用できるようにする
        value = (value - np.min(data)) / (np.max(data) - np.min(data)) * 2 - 1.0
        angle = 2 * np.arccos(value)
        if i < len(qubits):
            circuit.append(cirq.ry(angle)(qubits[i]))
    return circuit

In [5]:
def perform_qpca(qubits, circuits, num_iterations):
    """
    qPCAを実施し、エンコードされた量子データに対して次元削減を行います。
    """
    simulator = cirq.Simulator()
    density_matrices = []
    for circuit in circuits:
        state = simulator.simulate(circuit)
        density_matrix = np.outer(state.final_state_vector, np.conj(state.final_state_vector))
        density_matrices.append(density_matrix)

    # 全ての密度行列を平均して最終的な密度行列を作成
    avg_density_matrix = np.mean(density_matrices, axis=0)

    # 固有値と固有ベクトルを計算（qPCAの中心的な部分）
    eigenvalues, eigenvectors = eigh(avg_density_matrix)

    print("Eigenvalues:")
    print(eigenvalues)
    print("Eigenvectors:")
    print(eigenvectors)

    # 固有値が大きいものに基づいて次元削減
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_eigenvectors = eigenvectors[:, sorted_indices[:num_iterations]]

    return top_eigenvectors, eigenvalues, sorted_indices

In [6]:
def decode_quantum_data(top_eigenvectors, scaled_data):
    """
    qPCAで次元削減した量子データを古典データに低次元空間に射影します
    """
    # 元のデータを低次元空間に射影するために、データと固有ベクトルの次元を整合させる
    top_eigenvectors_reduced = top_eigenvectors[:scaled_data.shape[1], :]
    reduced_data = np.dot(scaled_data, top_eigenvectors_reduced)
    return reduced_data

In [7]:
def calculate_contribution_ratios(eigenvalues, sorted_indices, num_components):
    """
    寄与率を計算し、各主成分の寄与率の表を作成します。
    """
    total_variance = np.sum(eigenvalues)
    selected_eigenvalues = eigenvalues[sorted_indices[:num_components]]
    contribution_ratios = selected_eigenvalues / total_variance
    contribution_table = pd.DataFrame({
        'Component': [f'PC{i+1}' for i in range(num_components)],
        'Eigenvalue': selected_eigenvalues,
        'Contribution Ratio': contribution_ratios
    })
    return contribution_table

In [12]:
def main():
    # アイリスデータセットの読み込みと標準化
    scaled_data = load_and_scale_iris()

    # 量子ビットの準備
    num_qubits = scaled_data.shape[1]
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    # 各サンプルを量子エンコードし、回路を作成
    circuits = [create_quantum_encoding_circuit(qubits, data) for data in scaled_data]
    # 各サンプルを量子エンコードし、回路を作成
    # circuits = []
    # for data in scaled_data:
    #     circuits.append(create_quantum_encoding_circuit(qubits, data))
    #     # 複数のサンプルに対してより多くのエンコードを行うためにサンプル数を増やす
    #     for _ in range(5):  # 各サンプルに対して追加でエンコードを作成（例として5回）
    #         noisy_data = data + np.random.normal(0, 0.005, data.shape)  # ノイズの標準偏差を小さくして調整  # ノイズを加えて多様なサンプルを作成
    #         noisy_data = np.clip(noisy_data, -1.0, 1.0)
    #         noisy_data = np.nan_to_num(noisy_data)  # NaNが発生した場合にゼロに置き換える  # データを適切な範囲にクリップ
    #         noisy_data = np.nan_to_num(noisy_data)  # NaNが発生した場合にゼロに置き換える
    #         circuits.append(create_quantum_encoding_circuit(qubits, noisy_data))

    # 作成した各回路を表示
    for i, circuit in enumerate(circuits):
        print(f"Quantum Encoding Circuit for sample {i}:")
        print(circuit)

    # qPCAの実行
    top_eigenvectors, eigenvalues, sorted_indices = perform_qpca(qubits, circuits, num_iterations=2)
    print("Top Eigenvectors after qPCA:")
    print(top_eigenvectors)

    # qPCAで次元削減したデータを低次元空間に射影して古典データに戻す
    decoded_data = decode_quantum_data(top_eigenvectors, scaled_data)
    print("Decoded Classical Data:")
    print(decoded_data)

    # 寄与率の計算と表の作成
    contribution_table = calculate_contribution_ratios(eigenvalues, sorted_indices, num_components=2)
    print("Contribution Ratios Table:")
    print(contribution_table)

In [13]:
if __name__ == "__main__":
    main()

Quantum Encoding Circuit for sample 0:
0: ───Ry(1.43π)───

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(1.87π)───
Quantum Encoding Circuit for sample 1:
0: ───Ry(1.47π)───

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(1.82π)───
Quantum Encoding Circuit for sample 2:
0: ───Ry(1.9π)────

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(1.72π)───
Quantum Encoding Circuit for sample 3:
0: ───Ry(2π)──────

1: ───Ry(0)───────

2: ───Ry(1.51π)───

3: ───Ry(1.55π)───
Quantum Encoding Circuit for sample 4:
0: ───Ry(1.54π)───

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(1.87π)───
Quantum Encoding Circuit for sample 5:
0: ───Ry(1.4π)────

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(1.75π)───
Quantum Encoding Circuit for sample 6:
0: ───Ry(2π)──────

1: ───Ry(0)───────

2: ───Ry(1.65π)───

3: ───Ry(1.51π)───
Quantum Encoding Circuit for sample 7:
0: ───Ry(1.51π)───

1: ───Ry(0)───────

2: ───Ry(1.84π)───

3: ───Ry(2π)──────
Quantum Encoding Circuit for sample 8:
0: ───Ry(2π)─────